<a href="https://colab.research.google.com/github/Ruthicaf1/GroupProj/blob/main/pyspark_Rainfall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://se

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Advertising Spend").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/unit-7/AdvertisingSpend.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("AdvertisingSpend.csv"), sep=",", header=True)

# Show DataFrame
df.show()


+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|              Type|            Source|              2012|              2013|              2014|              2015|              2016|              2017|              2018|              2019|              2020|              2021|              2022|              2023|              2024|
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          Total TV|          Total TV|  	60,914,800,000 |  	62,895,700,000 |  	64,401,700,000 |  	65,382,100,000 |  	67,440,100,000 |  	66

In [29]:
def million_div(dollar):
  dollar = dollar.replace(",","")
  dollar_value=float(dollar)/1000000
  return dollar_value


In [46]:
def select_creator(column_array):
  query = "select Type, "
  for a in column_array:
    query = query + "million_div(`" + str(a) + "`), "
  query = query + "Source from DollarVersion"
  return query

In [47]:
query = select_creator([2012,2013,2014,2015,2016])

In [30]:
spark.udf.register("million_div", million_div)

<function __main__.million_div>

In [31]:
df.createOrReplaceTempView("DollarVersion")

In [48]:
temp_df = spark.sql(query)
temp_df.show()

+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+
|              Type|million_div(2012)|million_div(2013)|million_div(2014)|million_div(2015)|million_div(2016)|            Source|
+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+
|          Total TV|          60914.8|          62895.7|          64401.7|          65382.1|          67440.1|          Total TV|
|                TV|          38584.9|          39577.1|          41064.2|          41313.1|          43416.8|       National TV|
|                TV|          22329.9|          23318.6|          23337.5|          24069.0|          24023.3|          Local TV|
|             Radio|          16303.5|          16591.3|          16165.6|          16385.7|          16554.8|       Total Radio|
|        Newspapers|          24926.5|          23503.0|          21677.8|          20269.